# importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# importing necessary csv files 

In [2]:
df_ratings = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/ratings.csv")
df_movies = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/movies.csv", encoding  = 'unicode_escape') #contains the title
df_tags= pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/tags.csv") 
df_genome_scores = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/genome-scores.csv") 
df_links = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/links.csv")
df_genome_tags = pd.read_csv("D:/Spyder/Data-Envelopment-Analysis-Excel/Data/MOVIE PROJECT/ml-25m/ml-25m/genome-tags.csv")

# Checking each of them

### df_movies

In [3]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### links

In [4]:
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### ratings

In [5]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
df_ratings.shape

(25000095, 4)

In [7]:
df_ratings['userId'].value_counts().shape

(162541,)

#### out of 25000095 ratings done, 162541 are the unique raters. this is till a big number, so now we will narrow our search by removing all those users, who hadn't rated at least 500 movies

In [8]:
x = df_ratings['userId'].value_counts() > 500

In [9]:
y = x[x].index

In [10]:
y.shape

(9663,)

In [11]:
rating = df_ratings[df_ratings['userId'].isin(y)]

In [12]:
rating.shape

(8935061, 4)

In [13]:
rating

,userId,movieId,rating,timestamp
254,3,1,4.0,1439472215
255,3,29,4.5,1484754967
256,3,32,4.5,1439474635
257,3,50,5.0,1439474391
258,3,111,4.0,1484753849
...,...,...,...,...
24996936,162519,4748,2.0,1000947348
24996937,162519,4789,1.0,1000946127
24996938,162519,4855,5.0,1001884147
24996939,162519,4857,5.0,1001886067


### now merging df_movies and rating

In [14]:
movie_details = df_movies.merge(rating, on = 'movieId')

In [15]:
movie_details.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,1167582601
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,956264593
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,942014736
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,1074594380


In [16]:
movie_details.shape

(8935061, 6)

In [17]:
movie_details.drop(columns=['timestamp'], inplace = True)

In [18]:
movie_details.shape

(8935061, 5)

In [19]:
movie_details.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5


In [20]:
number_rating = movie_details.groupby('title')['rating'].count().reset_index()

In [21]:
number_rating.rename(columns = {'rating' : 'number of rating'}, inplace = True)

In [22]:
number_rating.head()

,title,number of rating
0,"""BLOW THE NIGHT!"" Let's Spend the Night Togeth...",1
1,"""Great Performances"" Cats (1998)",42
2,#1 Cheerleader Camp (2010),7
3,#Captured (2017),2
4,#Female Pleasure (2018),2


In [23]:
df_tillnow = movie_details.merge(number_rating, on = 'title')

In [24]:
df_tillnow.shape

(8935061, 6)

In [25]:
df_tillnow.head()

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,8078
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,8078
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,8078
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,8078
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,8078


#### by setting the threshold of 50 in number_of_rating so as to more narrow down our search space

In [26]:
df_tillnow1 = df_tillnow[df_tillnow['number of rating']>=50] #taking only the valuable one

In [27]:
df_tillnow1.shape

(8596657, 6)

In [28]:
df_tillnow1.head()

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,8078
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,8078
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,8078
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,8078
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,8078


#### now we don't need number_of_rating column

In [29]:
df_tillnow1.drop(columns=['number of rating'], inplace = True)

C:\Users\SHIVLEON\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
df_tillnow1.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5


In [31]:
df_tillnow1['rating'] = df_tillnow1['rating'].astype(float)

C:\Users\SHIVLEON\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_tillnow1.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5


In [33]:
movie_pivot = df_tillnow1.pivot_table(columns='title', index = 'userId', values = 'rating')

In [34]:
movie_pivot

title,'71 (2014),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC]Â² (2009),[REC]Â³ 3 GÃ©nesis (2012),eXistenZ (1999),iBoy (2017),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: Return of Xander Cage (2017),xXx: State of the Union (2005),Â¡Three Amigos! (1986),Ã nous la libertÃ© (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
72,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162495,NaN,NaN,NaN,NaN,3.5,NaN,NaN,3.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
162508,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
movie_pivot.shape

(9663, 11716)

In [36]:
movie_pivot.fillna(0, inplace = True)

In [37]:
from scipy.sparse import csr_matrix

In [38]:
movie_sparse = csr_matrix(movie_pivot)

In [39]:
movie_sparse

<9663x11716 sparse matrix of type '<class 'numpy.float64'>'
	with 8596205 stored elements in Compressed Sparse Row format>

In [40]:
from sklearn.neighbors import NearestNeighbors

In [41]:
model = NearestNeighbors(n_neighbors = 7, algorithm = 'brute', metric = 'cosine')

In [42]:
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                 radius=1.0)

In [43]:
distances, suggestions = model.kneighbors(movie_pivot[0:1].values.reshape(1, -1))

In [44]:
distances

array([[0.        , 0.41366313, 0.42033232, 0.42071414, 0.42077611,
        0.42142699, 0.42402189]])

In [45]:
suggestions

array([[   0, 7442, 8950, 3086, 9512, 5494, 7569]], dtype=int64)

In [46]:
movie_pivot[0:1]

title,'71 (2014),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC]Â² (2009),[REC]Â³ 3 GÃ©nesis (2012),eXistenZ (1999),iBoy (2017),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: Return of Xander Cage (2017),xXx: State of the Union (2005),Â¡Three Amigos! (1986),Ã nous la libertÃ© (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0


In [47]:
movie_pivot[7442:7443]

title,'71 (2014),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC]Â² (2009),[REC]Â³ 3 GÃ©nesis (2012),eXistenZ (1999),iBoy (2017),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: Return of Xander Cage (2017),xXx: State of the Union (2005),Â¡Three Amigos! (1986),Ã nous la libertÃ© (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
124561,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [48]:
movie_pivot.loc[3]

title
'71 (2014)                                     0.0
'Round Midnight (1986)                         0.0
'Salem's Lot (2004)                            0.0
'Til There Was You (1997)                      0.0
'burbs, The (1989)                             0.0
                                              ... 
xXx (2002)                                     3.5
xXx: Return of Xander Cage (2017)              0.0
xXx: State of the Union (2005)                 0.0
Â¡Three Amigos! (1986)                         0.0
Ã nous la libertÃ© (Freedom for Us) (1931)    0.0
Name: 3, Length: 11716, dtype: float64

In [49]:
distances, suggestions = model.kneighbors(movie_pivot.loc[3].values.reshape(1, -1))

In [50]:
distances

array([[0.        , 0.41366313, 0.42033232, 0.42071414, 0.42077611,
        0.42142699, 0.42402189]])

In [51]:
suggestions

array([[   0, 7442, 8950, 3086, 9512, 5494, 7569]], dtype=int64)

In [52]:
model1 = NearestNeighbors(n_neighbors = 11, algorithm = 'brute', metric = 'euclidean')

In [53]:
user3df = movie_pivot.loc[3].to_frame()

In [54]:
user3df

,3
title,
'71 (2014),0.0
'Round Midnight (1986),0.0
'Salem's Lot (2004),0.0
'Til There Was You (1997),0.0
"'burbs, The (1989)",0.0
...,...
xXx (2002),3.5
xXx: Return of Xander Cage (2017),0.0
xXx: State of the Union (2005),0.0


In [55]:
user3df.columns = ['rating']

In [56]:
user3df

,rating
title,
'71 (2014),0.0
'Round Midnight (1986),0.0
'Salem's Lot (2004),0.0
'Til There Was You (1997),0.0
"'burbs, The (1989)",0.0
...,...
xXx (2002),3.5
xXx: Return of Xander Cage (2017),0.0
xXx: State of the Union (2005),0.0


In [57]:
x1 = user3df['rating'].value_counts() > 2

In [58]:
x1

0.0    True
4.0    True
3.5    True
3.0    True
4.5    True
5.0    True
2.5    True
2.0    True
Name: rating, dtype: bool

In [59]:
y1 = x1[x1]

In [60]:
rateduser3df = user3df[user3df['rating']>1]

In [61]:
rateduser3df

,rating
title,
102 Dalmatians (2000),2.0
15 Minutes (2001),3.5
"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4.0
2001: A Space Odyssey (1968),5.0
2012 (2009),2.5
...,...
X2: X-Men United (2003),4.0
You Don't Mess with the Zohan (2008),3.0
Zombieland (2009),4.0


In [151]:
rateduser3df['rating'][0]

3.0

In [97]:
model1.fit(movie_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [99]:
distances1, similar_users1 = model1.kneighbors(movie_pivot.loc[3].values.reshape(1, -1))

In [100]:
distances1

array([[ 0.        , 80.11086069, 83.79588295, 84.76290462, 85.10728523,
        85.67234093, 85.69568251, 85.70443396, 85.82831701, 86.05085706]])

In [101]:
similar_users1

array([[   0, 3086,  197, 6327, 8712, 3293, 7970, 1767, 4399,  914]],
      dtype=int64)

In [108]:
distances1[0][0]

0.0

array([[ 0.        , 80.11086069, 83.79588295, 84.76290462, 85.10728523,
        85.67234093, 85.69568251, 85.70443396, 85.82831701, 86.05085706]])

In [109]:
list_index = rateduser3df.index.values.tolist()

In [111]:
list_index

['22 Jump Street (2014)',
 '27 Dresses (2008)',
 '28 Days (2000)',
 '28 Days Later (2002)',
 '40-Year-Old Virgin, The (2005)',
 '50 First Dates (2004)',
 'A-Team, The (2010)',
 'AVP: Alien vs. Predator (2004)',
 'About a Boy (2002)',
 'After Earth (2013)',
 'Analyze That (2002)',
 'Anger Management (2003)',
 'Astro Boy (2009)',
 'Babylon A.D. (2008)',
 'Bad Boys II (2003)',
 'Beach, The (2000)',
 'Bedazzled (2000)',
 "Bill & Ted's Excellent Adventure (1989)",
 'Blade II (2002)',
 'Break-Up, The (2006)',
 "Bridget Jones's Diary (2001)",
 'Bridget Jones: The Edge of Reason (2004)',
 'Bright (2017)',
 'Bruce Almighty (2003)',
 'Catwoman (2004)',
 'Cell, The (2000)',
 'Click (2006)',
 'Dodgeball: A True Underdog Story (2004)',
 'Don Jon (2013)',
 'Due Date (2010)',
 'Elektra (2005)',
 'Evolution (2001)',
 'Expendables, The (2010)',
 'Flightplan (2005)',
 'Focus (2015)',
 'Friends with Benefits (2011)',
 'Furious 7 (2015)',
 'G.I. Joe: The Rise of Cobra (2009)',
 'Get Smart (2008)',
 'Girl 

In [116]:
new_dataframe = movie_pivot.iloc[similar_users1[0][1]].to_frame()

In [117]:
new_dataframe

,51328
title,
'71 (2014),0.0
'Round Midnight (1986),0.0
'Salem's Lot (2004),0.0
'Til There Was You (1997),0.0
"'burbs, The (1989)",1.5
...,...
xXx (2002),1.5
xXx: Return of Xander Cage (2017),0.0
xXx: State of the Union (2005),0.0


In [118]:
new_dataframe.columns = ['rating']

In [119]:
new_dataframe

,rating
title,
'71 (2014),0.0
'Round Midnight (1986),0.0
'Salem's Lot (2004),0.0
'Til There Was You (1997),0.0
"'burbs, The (1989)",1.5
...,...
xXx (2002),1.5
xXx: Return of Xander Cage (2017),0.0
xXx: State of the Union (2005),0.0


In [120]:
new_dataframe1 = movie_pivot.iloc[similar_users1[0][2]].to_frame()

In [121]:
new_dataframe1.columns = ['rating']

In [122]:
added_dataframe = new_dataframe.add(new_dataframe1, fill_value = 0)

In [123]:
added_dataframe

,rating
title,
'71 (2014),0.0
'Round Midnight (1986),0.0
'Salem's Lot (2004),0.0
'Til There Was You (1997),0.0
"'burbs, The (1989)",1.5
...,...
xXx (2002),1.5
xXx: Return of Xander Cage (2017),0.0
xXx: State of the Union (2005),0.0


In [124]:
new_dataframe2 = new_dataframe[new_dataframe1.index.isin(['xXx (2002)', 'xXx: Return of Xander Cage (2017)'])]

In [125]:
new_dataframe2

,rating
title,
xXx (2002),1.5
xXx: Return of Xander Cage (2017),0.0


In [62]:
modelr = NearestNeighbors(n_neighbors = 11, algorithm = 'brute', metric = 'euclidean')
modelr.fit(movie_sparse)
def recommend(user):
    #lets get the similar users
    distances, similar_users = modelr.kneighbors(movie_pivot.loc[user].values.reshape(1, -1))
    #making the user's row as an dataframe itself for proper use
    userdf = movie_pivot.loc[user].to_frame()
    userdf.columns = ['rating'] #there are two columns, title as index, rating the 2nd column
    """print('userdf\n')
    print(userdf.head())
    print('\n') """
    
    #now getting those movies which are rated by the by the user itself, as we don't want to suggest it again
    #X = userdf['rating'].value_counts() > 1
    #Y = X[X]
    rated_by_user = userdf[userdf['rating']>1]
    print('rated_by_user\n')
    print(rated_by_user.head(10))
    print('\n')
    
    #getting the index of rated_by_user as a list, as it contains the name of the movies which user had already watched
    watched_already = rated_by_user.index.values.tolist()
    
    #now combining all the similar users rating, by making the index common
    #by using loop, we are gonna form new dataframe and keep adding them
    #1st making a new dataframe, with already 1st user in it
    
    other_user_df = movie_pivot.iloc[similar_users[0][1]].to_frame()
    other_user_df.columns = ['rating'] #there are two columns, title as index, rating the 2nd column
    for i in range(2,len(similar_users)):
        subdf = movie_pivot.iloc[similar_users[0][i]].to_frame()
        subdf.columns = ['rating']
        other_user_df = other_user_df.add(subdf, fill_value = 0)
    
    
    #there still be many movies which all the rest of the users wouldn't have watched as well
    #lets get rid of them
    #Xo = other_user_df['rating'].value_counts() > 1
    #Yo = Xo[Xo]
    rated_by_other_users = other_user_df[other_user_df['rating']>1]
    """print('rated_by_other_users\n')
    print(rated_by_other_users.head(10))
    print('\n')"""
    
    #now that we have filtered dataframe where the movies wathed by others is there,
    #we have to get rid of those movies, which the original user had already watched
    #here, watched_already list will help us out so as to this 
    
    filtered_rated_by_other_users = rated_by_other_users[~rated_by_other_users.index.isin(watched_already)]
    """print('filtered_rated_by_other_users\n')
    print(filtered_rated_by_other_users.head(10))
    print('\n')"""
    
    #now as we have a list of unwatched movies by our given users,
    #now we are gonna normalize our ratings so that the ratings is b/w 0-1
    
    filtered_rated_by_other_users['rating'] = filtered_rated_by_other_users['rating'].div(len(similar_users[0]) - 1)
    
    
    #now we are gonna sort the values in descending order so that we can get the highest rated movies combined by the other users
    recommendation_to_user = filtered_rated_by_other_users.sort_values('rating', ascending = False)
    print('recommendation_to_user\n')
    print(recommendation_to_user.head(10))

In [63]:
recommend(3)

rated_by_user

                                                    rating
title                                                     
102 Dalmatians (2000)                                  2.0
15 Minutes (2001)                                      3.5
2 Fast 2 Furious (Fast and the Furious 2, The) ...     4.0
2001: A Space Odyssey (1968)                           5.0
2012 (2009)                                            2.5
22 Jump Street (2014)                                  3.0
25th Hour (2002)                                       4.5
27 Dresses (2008)                                      3.0
28 Days (2000)                                         3.0
28 Days Later (2002)                                   3.0


recommendation_to_user

                                                    rating
title                                                     
Spirited Away (Sen to Chihiro no kamikakushi) (...    0.50
Planet Earth II (2016)                                0.50
Dunkirk (2017) 

C:\Users\SHIVLEON\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [64]:
recommend(72)

rated_by_user

                                                    rating
title                                                     
'burbs, The (1989)                                     3.0
10 Things I Hate About You (1999)                      5.0
101 Dalmatians (One Hundred and One Dalmatians)...     3.0
12 Angry Men (1957)                                    5.0
200 Cigarettes (1999)                                  3.0
2001: A Space Odyssey (1968)                           3.0
2010: The Year We Make Contact (1984)                  4.0
28 Days (2000)                                         4.0
8MM (1999)                                             3.0
Abyss, The (1989)                                      4.0


recommendation_to_user

                                              rating
title                                               
Peeping Tom (1960)                               0.4
To Kill a Mockingbird (1962)                     0.4
Kids (1995)                            

C:\Users\SHIVLEON\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
recommend(120)

rated_by_user

                                                    rating
title                                                     
10 Things I Hate About You (1999)                      5.0
101 Dalmatians (One Hundred and One Dalmatians)...     4.0
2001: A Space Odyssey (1968)                           5.0
28 Days (2000)                                         4.0
6th Day, The (2000)                                    4.0
9 1/2 Weeks (Nine 1/2 Weeks) (1986)                    4.0
Abyss, The (1989)                                      5.0
Action Jackson (1988)                                  4.0
Adventures in Babysitting (1987)                       4.0
Adventures of Buckaroo Banzai Across the 8th Di...     4.0


recommendation_to_user

                                                    rating
title                                                     
Chinatown (1974)                                       0.5
Lord of the Rings: The Fellowship of the Ring, ...     0.5
'burbs, The (19

C:\Users\SHIVLEON\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
